In [1434]:
import sys
sys.path.append("../")
import nanonis
import numpy as np
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
import xlrd
import lmfit
from lmfit import Model
import matplotlib.colors as colors
from scipy import constants as const
%matplotlib qt
spectra=nanonis.biasSpectroscopy()

# Useful functions

In [1082]:
def fermi(x,T):
    return np.divide(1,1+np.exp(x/T))


def BCS(x,delta,eta):
    return np.sign(x)*np.imag(np.divide(np.abs(x+1j*eta),np.sqrt(delta**2-(x+1j*eta)**2)))

def coulomb(x,Ec,Q0):
    return np.heaviside(x-Ec-Q0,0)+np.heaviside(-x-Ec+Q0,0)

def conv_dynes(x,delta1,delta2,eta,T,Ec,Q0):
    y=np.linspace(-5*(delta1+delta2),5*(delta1+delta2),5000)
    a,b=np.meshgrid(y,y)
    cou=np.gradient(-np.dot(BCS(a+b,delta1,eta)*fermi(a+b,T),coulomb(y,Ec,Q0))+np.dot(BCS(a+b,delta1,eta),fermi(y,T)*coulomb(y,Ec,Q0)))
    a,b=np.meshgrid(y,x)
    z=np.gradient(-np.dot(BCS(a+b,delta2,eta)*fermi(a+b,T),cou)+np.dot(BCS(a+b,delta2,eta),cou*fermi(y,T)))
    return z/z[0]

def temp(x,delta1,delta2,eta,T):
    y=np.linspace(-5*(delta1+delta2),5*(delta1+delta2),5000)
    a,b=np.meshgrid(y,x)
    return -np.dot(BCS(a+b,delta2,eta)*fermi(a+b,T),BCS(y,delta1,eta))+np.dot(BCS(a+b,delta2,eta),BCS(y,delta1,eta)*fermi(y,T))

def temp2(V,delta1,delta2,eta,T):
    y=np.linspace(-5*(delta1+delta2),5*(delta1+delta2),5000)
    return np.sum(-BCS(y+V,delta2,eta)*fermi(y+V,T)*BCS(y,delta1,eta)+BCS(y+V,delta2,eta)*BCS(y,delta1,eta)*fermi(y,T))

def gap(x,delta1,Bc1,Ec,a):
    d1=(0.0013*np.real(np.sqrt(1-((x+0.0000*1j)/800)**2)))*np.heaviside(-x+800,0)
    d2=delta1*np.real(np.sqrt(1-((x+0.0000*1j)/Bc1)**2))*np.heaviside(-x+Bc1,0)
    d3=(-a*x)*np.heaviside(x-Bc1,0)
    return Ec+d1+d2+d3
   
def coulomb_T(x,Ec,T):
    y=np.linspace(-3*np.min(x),3*np.min(x),2000)
    a,b=np.meshgrid(y,x)
    return np.gradient(-np.dot(fermi(a+b,T),coulomb(y,Ec,Q0))+fermi(y,T)*coulomb(y,Ec,Q0))

def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx


# File management #

In [1057]:
data=pd.read_excel("C:/Users/jonor/Documents/GitHub/nanonis/Order.xls")

In [1079]:
Bfield_files=data.iloc[:,2]
Bfields=data.iloc[:,3]
size_dat=data.iloc[:,7]
height_dat=data.iloc[:,8]
temp=data.iloc[:,6]
temp_files=data.iloc[:,5]
josephson=data.iloc[:,4]
path=data.iloc[:,10]

# B Field 

In [60]:
## All files
dat=[1,2,3,4,5,6,7,8,10,11,12,13,14,15,16,17,18,19,21,23,24]
BFields=[]
BFiles=[]
size=[]
height=[]
for i in dat:
    a=[int(j) for j in Bfields[i].split(',')]
    b=Bfield_files[i].split(',')    
    files=[path+i for i in b]
    BFields.append(a)
    BFiles.append(files)
    size.append(size_dat[i])
    height.append(height_dat[i])
size=np.array(size)
height=np.array(height)

In [734]:
i=12
a=[int(j) for j in Bfields[i].split(',')]
b=Bfield_files[i].split(',')
files=[path+i for i in b]


In [154]:
w_B=[0,1,2,3,4,5,6,7,8,10,11,12,13,14,15,16,17,18,19,21,23,24]
Bfiles=[]
size=[]
B=[]

for j in w_B:
    a=[int(i) for i in Bfields[j].split(',')]
    b=Bfield_files[j].split(',')
    files=[path+i for i in b]
    B.append(a)
    #field0=a.index(0)
    #files_B0.append(files[field0])
    Bfiles.append(files)
    size.append(size_dat[j])


### Plots

In [722]:
y=[]
for i in files:
    spectra.load(i)
    y.append(spectra.conductance/spectra.conductance[0])
x=np.array(spectra.bias)
plt.imshow(y,aspect='auto',origin='lower',extent=(1000*x[0],1000*x[-1],a[-1],a[0]))
plt.ylabel('B(mT)')
plt.xlabel('Bias (meV)')

Text(0.5, 0, 'Bias (meV)')

### Some info

We look at the B field point after 800 mT to see the effect of B field on the coulomb gap

In [216]:
def coulomb_T(x,Ec,Q0,T):
    return fermi(Ec-x+Q0,T)+fermi(Ec+x-Q0,T)
def coulomb(x,Ec,Q0):
    T=0.0003
    return fermi(Ec-x+Q0,T)+fermi(Ec+x-Q0,T)
def coulomb_T2(x,Ec,Q0,T):
    return np.sinh(np.abs(x)/T)/(np.cosh(x/T)+np.cosh(Ec/T))

h=0
Bfield=[800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300,2400,2500]
Gap=[]
Asym=[]
Si=[]
for k in Bfield:
    Ec=[]
    Q0=[]
    size2=[]
    for i in range(len(Bfiles)):
        for j in range(len(Bfiles[i])):
            if np.abs(B[i][j]-k)<=0.00001:
                size2.append(size[i])
                spectra.load(Bfiles[i][j])
                x=np.array(spectra.bias)
                y=np.array(spectra.conductance)
                y=y/y[0]
                #values=sc.optimize.curve_fit(coulomb_T,x,y,p0=[0.001,0.00,0.0003])[0]
                values=sc.optimize.curve_fit(coulomb,x,y,p0=[0.001,0.00])[0]
                #yy=coulomb_T(x,values[0],values[1],values[2])
                #yy=coulomb(x,values[0],values[1])
                Ec.append(values[0])
                Q0.append(values[1])
                #ther.append(values[2])
                #plt.plot(x,yy+h)
                #plt.plot(x,y+h)
                #h+=1
            else:
                continue
    Gap.append(Ec)
    Asym.append(Q0)
    Si.append(size2)

In [411]:
def coulomb_T(x,Ec,Q0,T):
    return fermi(Ec-x+Q0,T)+fermi(Ec+x-Q0,T)
def coulomb(x,Ec,Q0):
    T=0.0003
    return fermi(Ec-x+Q0,T)+fermi(Ec+x-Q0,T)
def coulomb_T2(x,Ec,Q0,T):
    return np.sinh(np.abs(x)/T)/(np.cosh(x/T)+np.cosh(Ec/T))

h=0
Bfield=[800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300,2400,2500]
Gap=[]
Asym=[]
Si=[]
for k in Bfield:
    Ec=[]
    Q0=[]
    size2=[]
    for i in range(len(Bfiles)):
        for j in range(len(Bfiles[i])):
            if np.abs(B[i][j]-k)<=0.00001:
                size2.append(size[i])
                spectra.load(Bfiles[i][j])
                x=np.array(spectra.bias)
                y=np.array(spectra.current)
                y=y/y[0]
                xx=[]
                for p in range(len(y)):
                    if np.abs(y[p]/y[0])<0.03:
                        xx.append(x[p])
                Ec.append(np.array(xx)[0]-np.array(xx)[-1])
            else:
                continue
        
    Gap.append(Ec)
    Asym.append(Q0)
    Si.append(size2)

In [412]:

plt.scatter(Si[4],Gap[4])


In [183]:
def coulomb(x,Ec,Q0):
    T=0.0003
    return fermi(Ec-x+Q0,T)+fermi(Ec+x-Q0,T)


emp=[]
for i in Bfiles:
    en=[]
    for j in i:
        en.append(0.0)
    emp.append(en)
Ec=emp
Q0=emp
for i in range(len(Bfiles)):
    for j in range(len(Bfiles[i])):
        spectra.load(Bfiles[i][j])
        x=np.array(spectra.bias)
        y=np.array(spectra.conductance)
        y=y/y[0]
        values=sc.optimize.curve_fit(coulomb,x,y,p0=[0.001,0.00])[0]
        #Ec[i][j]=values[0]
        Q0[i][j]=values[1]
        Ec[i][j]=values[0]
        
        


In [413]:
emp=[]
for i in Bfiles:
    en=[]
    for j in i:
        en.append(0.0)
    emp.append(en)
Ec=emp
Q0=emp
for i in range(len(Bfiles)):
    for j in range(len(Bfiles[i])):
        spectra.load(Bfiles[i][j])
        x=np.array(spectra.bias)
        y=np.array(spectra.current)
        y=y/y[0]
        xx=[]
        for p in range(len(y)):
            if np.abs(y[p]/y[0])<0.01:
                xx.append(x[p])
        Ec[i][j]=np.array(xx)[0]-np.array(xx)[-1]        

In [ ]:
i=4
plt.plot(xx[i],yy[i])

In [420]:
xx=[]
yy=[]
for j in range(22):
    x=[]
    y=[]
    si=size[j]
    for i in range(len(B[j])):
        if B[j][i]>=1000:
            x.append(B[j][i])
            y.append(Ec[j][i])
        else:
            continue
    xx.append(x)
    yy.append(y)
    a,b=np.polyfit(x,y,1)
    plt.plot(si,a,'o')

In [416]:

plt.plot(xx[4],yy[4])

In [241]:
plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.viridis(np.linspace(0,1,10)))
#for i in range(18):
#    plt.scatter(Si[:][i],1000*np.array(Asym[:][i]))
plt.scatter(Si[0],1000*np.array(Asym[0]))
plt.scatter(Si[17],1000*np.array(Asym[17]))


#plt.scatter(size2,1000*np.array(Q08),c='#ff7f0e',label='T=8K')
#plt.xlabel('Island Area ($nm^2$)')
#plt.ylabel('$Q_0$ ($meV$)')
#plt.legend()

In [234]:
np.sum(np.abs(0-np.array(Asym[0])))/len(np.array(Asym[0])),np.sum(np.abs(0-np.array(Asym[17])))/len(np.array(Asym[17]))

(0.00021199756024640996, 0.00012050366485309387)

### Some info

The simplest way to get the gao is to calculate it from the current. We take the points for which the current is zero.



In [61]:
h=0
Ec=[]
Eg=[]
for k in range(len(BFiles)):
    cur=[]
    for i in BFiles[k]:
        spectra.load(i)
        cur.append(spectra.current)

    t=[]
    for j in range(len(BFiles[k])):
        y=np.array(cur[j])
        x=np.array(spectra.bias)
        xx=[]
        for i in range(len(y)):
            if np.abs(y[i]/y[0])<0.04:
                xx.append(x[i])
        t.append(xx[0]-xx[-1])
    T800=BFields[k].index(800)
    T0=BFields[k].index(0)
    Ec.append(t[T0])


plt.plot(size,Ec,'bo')


### Solme info

The critical field for the tip is around B=800mT, while the critical field for the island should be around B=350mT. Althought the first statement is prety robust, the second is not. 

The following program fit the spectra with a simple convolution. The free parameters will be the coulomb gap (Ec) the shift (Q0) and the dynes parameter (eta). We will suppose that the gaps decrease following the BCS theory with the critical fields that we wrote in the previous line. This first program is not a realistic fir of the system, we will only use it to get an idea of the volution of the full gap of the measurement, i.e., E+Delta1+Delta2. We will not make assupmtions about any of this parameters from the results of this analysis.

In [ ]:

def delta(Delta,B,Bc):
    return Delta*np.sqrt(1-(B/Bc)**2)

def conv_dynes(x,delta,eta,Ec):
    T=delta/10
    Q0=-0.00025
    y=np.linspace(-10*delta,10*delta,2000)
    a,b=np.meshgrid(y,x)
    z=np.gradient(-np.dot(BCS(a+b,delta,eta)*fermi(a+b,T),coulomb(y,Ec,Q0))+np.dot(BCS(a+b,delta,eta),fermi(y,T)*coulomb(y,Ec,Q0)))
    return z/z[0]


cond=[]
for i in files:
    spectra.load(i)
    cond.append(spectra.conductance)

gap=[]
for j in range(0,10):
    y=np.array(cond[j])[110:290]/np.array(cond[j])[110]
    x=np.array(spectra.bias)[110:290]
    temp=fit(conv_dynes,x,y,p0=[0.0013,0.0001,0.0012])
    plt.plot(x,y+0.5*j)
    p=temp[0]
    z=conv_dynes(x,p[0],p[1],p[2])
    plt.plot(x,z+0.5*j)
    gap.append(p[0]+p[2])
    print(j)

# Temperature dependence

In [421]:
data=[8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]
T=[]
files=[]
size=[]
height=[]

for j in data:
    a=temp[j].split(',')
    T.append([float(i) for i in a])
    b=temp_files[j].split(',')
    files.append([path+i for i in b])
    size.append(size_dat[j])
    height.append(height_dat[j])

In [422]:
f=files[0][0]
spectra.load(f)

x=np.array(spectra.bias)
y=np.array(spectra.conductance)
y=y/np.sum(y)


In [ ]:
x=np.linspace(-0.01,0.01,1000)
plt.plot(x,coulomb_T(x,0.0005,0.0))

In [149]:
def coulomb_T(x,Ec,Q0,T):
    return fermi(Ec-x+Q0,T)+fermi(Ec+x-Q0,T)
def coulomb(x,Ec,Q0):
    T=0.0019455504685344104
    return fermi(Ec-x+Q0,T)+fermi(Ec+x-Q0,T)
def coulomb_T2(x,Ec,Q0,T):
    return np.sinh(np.abs(x)/T)/(np.cosh(x/T)+np.cosh(Ec/T))
Ec=[]
Q0=[]
ther=[]
h=0
temperature=8.0
size2=[]
for i in range(len(files)):
    for j in range(len(files[i])):
        if np.abs(T[i][j]-temperature)<=0.00001:
            size2.append(size[i])
            spectra.load(files[i][j])
            x=np.array(spectra.bias)
            y=np.array(spectra.conductance)
            y=y/y[0]
            #values=sc.optimize.curve_fit(coulomb_T,x,y,p0=[0.001,0.00,0.002])[0]
            values=sc.optimize.curve_fit(coulomb,x,y,p0=[0.001,0.00])[0]
            #yy=coulomb_T(x,values[0],values[1],values[2])
            yy=coulomb(x,values[0],values[1])
            Ec.append(values[0])
            Q0.append(values[1])
            #ther.append(values[2])
            #plt.plot(x,yy+h)
            #plt.plot(x,y+h)
            h+=1
        else:
            continue
        


plt.scatter(size2,1000*np.array(Ec),c='#ff7f0e',label='T=7K')

In [145]:
np.mean(ther)

0.0019455504685344104

In [177]:
Ec7
size7
size8
Ec8
Q08
Q07

In [153]:
plt.scatter(size7,1000*np.array(Q07),c='#1f77b4',label='T=7K')
x=np.linspace(100,2000,1000)
#plt.plot(x,1/(0.001*x))
plt.scatter(size8,1000*np.array(Q08),c='#ff7f0e',label='T=8K')
plt.xlabel('Island Area ($nm^2$)')
plt.ylabel('$E_C$ ($meV$)')
plt.legend()
#plt.scatter(np.array(size),1000*np.array(Q0),c='#ff7f0e')
#plt.scatter(np.array(size),1000*np.array(ther),c='#2ca02c')

In [175]:
np.sum(np.array(ther))/len(ther)

0.002530124760784503

# Josephson 

In [1111]:
Jos_files=[]
JSize=[]
a=[0,1,2,3,4,5,6,7,8,11,12,13,14,15,16,17,18,19,21,22,23,24,25,26,27,28,29,30,31,32,33]
for j in a:
    Jos_files.append(np.array([path[j]+i for i in josephson[j].split(',')]))
    JSize.append(size_dat[j])
JSize, Jos_files = (list(t) for t in zip(*sorted(zip(JSize,Jos_files))))

In [1342]:
fig, axs = plt.subplots(5,7)
h=0
j=0
for i in range(len(Jos_files)):
    spectra.load(Jos_files[i][-1])
    x=np.array(spectra.bias)
    y=np.array(spectra.current)
    #yt=np.sum(y)
    #y=y/yt
    a1=min(find_nearest(x,-0.0012),find_nearest(x,0.0012))
    a2=max(find_nearest(x,-0.0012),find_nearest(x,0.0012))
    axs[j,h].plot(1000*x[a1:a2],y[a1:a2],label=JSize[i])
    #axs[j,h].plot(1000*x,y,label=JSize[i])
    h+=1
    if h==7:
        h=0
        j+=1


In [1718]:
ax1 = plt.subplot2grid((2,8), (0,0), colspan=2)
ax2 = plt.subplot2grid((2,8), (0,3), colspan=2)
ax3 = plt.subplot2grid((2,8), (0,6), colspan=2)
ax4 = plt.subplot2grid((2,8), (1,2), colspan=2)
ax5 = plt.subplot2grid((2,8), (1,5), colspan=2)
ax=[ax1,ax2,ax3,ax4,ax5]
h=0
for i in  [0,1,2,4]:
    spectra.load(Jos_files[i][-1])
    x=np.array(spectra.bias)
    y=np.array(spectra.current)
    x0=float(spectra.header.get('Lock-in>Amplitude'))
    print(x0)
    #yt=np.sum(y)
    #y=y/yt
    #ax[h].plot(1000*x,y/10e-9,label=JSize[i],linewidth=3)
    ax[h].set_xlabel('Bias (meV)')
    ax[h].set_ylabel('Current (nA)')
    a1=min(find_nearest(x,-0.0015),find_nearest(x,0.0015))
    a2=max(find_nearest(x,-0.0015),find_nearest(x,0.00150))
    x=x[a1:a2]
    y=y[a1:a2]
    ax[h].plot(x,y/10e-9,label=JSize[i],linewidth=3)
    ax[h].vlines(x0,np.min(y)/10e-9,np.max(y)/10e-9)
    ax[h].vlines(-x0,np.min(y)/10e-9,np.max(y)/10e-9)
    ax[h].axhline(0.0)
    #axs[j,h].plot(1000*x,y,label=JSize[i])
    h+=1

0.0001
0.0001
5e-05
0.0001


In [1735]:
sure=[1,2,4,6,18]
meh=[0,8]

[2, 4, 6, 8]

In [1774]:
spectra.load(Jos_files[30][-1])
x=np.array(spectra.bias)
y=np.array(spectra.current)
#x0=float(spectra.header.get('Lock-in>Amplitude'))
a1=min(find_nearest(x,-0.0005),find_nearest(x,0.0005))
a2=max(find_nearest(x,-0.0005),find_nearest(x,0.0005))
y=y[a1:a2]
x=x[a1:a2]
plt.plot(x,y)
#plt.vlines(0,np.min(y),np.max(y))
plt.vlines(x0,np.min(y),np.max(y))
plt.vlines(-x0,np.min(y),np.max(y))
plt.axhline(0.0)


In [1678]:
spectra.load(Jos_files[18][1])
x=np.array(spectra.bias)
y=np.array(spectra.current)
plt.plot(x,y)
plt.axhline(0.0)
plt.vlines(100e-6,np.min(y),np.max(y))
plt.vlines(-100e-6,np.min(y),np.max(y))



In [1679]:
Jos_files[0][-1]

'C:/Users/jonor/Desktop/PhD/Lanak/Pb on Gr (analysis)/2023.07.26 - 1st evap/S230802_246.dat'

In [1659]:
current0=[]
resistance=[]
a=range(len(Jos_files))
for i in a:
    spectra.load(Jos_files[i][-1])
    x=np.array(spectra.bias)
    y=np.array(spectra.current)
    #yt=np.sum(y)
    #y=y/yt
    a1=find_nearest(x,0.0)
    current0.append(y[a1])
    resistance.append(np.mean(x[len(x)-10:len(x)]/y[len(x)-10:len(x)]))
    #axs[j,h].plot(1000*x,y,label=JSize[i])

plt.scatter(a,np.array(current0)/10e-12)




# Islans by island

In [525]:
dat=[ 8, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 23, 24]
T=[]
Tfiles=[]
Bfiles=[]
Jfiles=[]
size=[]
height=[]
Bfiles=[]
B=[]

for j in dat:

    T.append([float(i) for i in temp[j].split(',')])

    Tfiles.append([path+i for i in temp_files[j].split(',')])

    size.append(size_dat[j])

    height.append(height_dat[j])

    B.append([int(i) for i in Bfields[j].split(',')])

    Bfiles.append([path+i for i in Bfield_files[j].split(',')])

    Jfiles.append([path+i for i in josephson[j].split(',')])




In [684]:
spectra.load(Jfiles[9][0])
x=np.array(spectra.bias)
y=np.array(spectra.conductance)
y=y/np.sum(y)
#x=x[150:len(x)-150]
#y=y[150:len(y)-150]
y=sc.signal.savgol_filter(y,7,1)
peak=sc.signal.find_peaks(y,threshold=0.0000001,width=3,height=0.005)[0]
plt.plot(x,y)
plt.scatter(x[peak],y[peak])
print(x[peak[0]],x[peak[1]],y[peak[0]],y[peak[1]],x[peak[0]]-x[peak[1]])

spectra.load(Jfiles[9][1])
x=np.array(spectra.bias)
y=np.array(spectra.conductance)
y=y/np.sum(y)
x=x[120:len(x)-120]
y=y[120:len(y)-120]
y=sc.signal.savgol_filter(y,7,1)
peak=sc.signal.find_peaks(y,threshold=0.0000001,width=3,height=0.0000001)[0]
plt.plot(x,y)
plt.scatter(x[peak],y[peak])
x[peak[0]],x[peak[1]],y[peak[0]],y[peak[1]],x[peak[0]]-x[peak[1]]

0.00251503 -0.00251503 0.0071214537653132666 0.005041787143825275 0.00503006


(0.00249499,
 0.000270541,
 0.006320886740342435,
 8.147776257827035e-05,
 0.002224449)

In [674]:
8.911575699453035e-05- 0.00008911575699453035

0.0

In [854]:
C1=[]
Q0=[]
ther=[]
C2=[]
h=0
for i in range(len(dat)):
    spectra.load(Tfiles[i][T[i].index(8.0)])
    x=np.array(spectra.bias)
    y=np.array(spectra.conductance)
    y=y/np.sum(y)
    plt.plot(1000*x,y+0.005*h)
    par=sc.optimize.curve_fit(lambda xx, C1,Q0,T,C2: current(xx, 1,10000, C1,C2,Q0,T), 1000*x, y,p0=[0.9,0.0,1,0.001],bounds=([0.1,-0.45,0.64,0.0005],[1.0,0.45,1.5,0.001]))[0]
    a=current(1000*x,1,10000,par[0],par[3],par[1],par[2])
    plt.plot(1000*x,a+0.005*h)
    h+=1
    C1.append(par[0])
    Q0.append(par[1])
    ther.append(par[2])
    C2.append(par[3])


In [887]:
C1=[]
Q0=[]
ther=[]
C2=[]
h=0
for i in range(len(dat)):
    spectra.load(Tfiles[i][T[i].index(8.0)])
    x=np.array(spectra.bias)
    y=np.array(spectra.conductance)
    y=y/np.sum(y)
    plt.plot(1000*x,y+0.005*h)
    par=sc.optimize.curve_fit(lambda xx, C1,Q0,C2: current(xx, 1,10000, C1,C2,Q0,0.64), 1000*x, y,p0=[0.9,0.0,0.001],bounds=([0.1,-0.45,0.0005],[1.0,0.45,0.1]))[0]
    a=current(1000*x,1,10000,par[0],par[2],par[1],0.64)
    plt.plot(1000*x,a+0.005*h)
    h+=1
    C1.append(par[0])
    Q0.append(par[1])
    #ther.append(par[2])
    C2.append(par[2])


In [861]:
plt.scatter(size,C1)

In [ ]:
ther=[]
Delta_island=[]
Q0=[]
h=0
for i in range(len(dat)):
    print(i)
    spectra.load(Tfiles[i][T[i].index(1.3)])
    x=np.array(spectra.bias)
    y=np.array(spectra.conductance)
    y=y/np.sum(y)
    plt.plot(1000*x,y+0.005*h)
    par=sc.optimize.curve_fit(lambda xx, Q0,T,Delta: currentD(xx, 1,10000, C1[i],C2[i],Q0,Delta,T), 1000*x, y,p0=[0.9,0.0,1.0],bounds=([0.1,-0.45,0.01],[1.0,0.45,1.3]))[0]
    a=currentD(1000*x,1,10000, C1[i],C2[i],par[0],par[2],par[1])
    plt.plot(1000*x,a+0.005*h)
    h+=1
    Q0.append(par[0])
    Delta_island.append(par[2])
    ther.append(par[1])


In [486]:
y=[]
for i in Bfiles[1]:
    spectra.load(i)
    y.append(spectra.conductance/spectra.conductance[0])
x=np.array(spectra.bias)
plt.imshow(y,aspect='auto',origin='lower',extent=(1000*x[0],1000*x[-1],a[-1],a[0]))
plt.ylabel('B(mT)')
plt.xlabel('Bias (meV)')

Text(0.5, 0, 'Bias (meV)')

## Low impedance

In [1209]:
def fermi(T,x):
    return np.divide(1,1+np.exp(x/T))

def E1(C1,C2,V,n,Q0):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def E2(C1,C2,V,n,Q0):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def Gamma1(V,R1,C1,C2,n,Q0,T):
    k=C2/(C1+C2)
    Ec=(C1+C2)/(2*C1*C2)
    x=E1(C1,C2,V,n,Q0)
    return (x/(1-np.exp(-x/T)))/R1

def Gamma2(V,R2,C1,C2,n,Q0,T):
    k=C1/(C1+C2)
    Ec=(C1+C2)/(2*C1*C2)
    x=E2(C1,C2,V,-n,-Q0)
    return (x/(1-np.exp(-x/T)))/R2


def PN(V,R1,R2,C1,C2,Q0,T,n):
    mn=(Gamma1(V,R1,C1,C2,-n,Q0,T)+Gamma2(-V,R2,C1,C2,n,-Q0,T))/(Gamma1(-V,R1,C1,C2,n+1,-Q0,T)+Gamma2(V,R2,C1,C2,-n-1,Q0,T))
    en=(Gamma1(-V,R1,C1,C2,-n,-Q0,T)+Gamma2(V,R2,C1,C2,n,Q0,T))/(Gamma1(V,R1,C1,C2,n+1,Q0,T)+Gamma2(-V,R2,C1,C2,-n-1,-Q0,T))
    return en,mn

def check_p(V,R1,R2,C1,C2,Q0,T):
    Vmax=V
    n=[0]
    while True:
        a=[]
        b=[]
        for i in n:
            an,bn=PN(Vmax,R1,R2,C1,C2,Q0,T,i)
            a.append(an[0])
            b.append(bn[0])
        p0=1
        for i in range(len(a)):
            temp=np.zeros(len(a))
            temp[0:i+1]=1
            temp=temp.tolist()
            p0+=np.prod(a,where=temp)+np.prod(b,where=temp)
        p0=1/p0
        pn=p0*np.prod(a)
        p_n=p0*np.prod(b)
        #print(pn,p_n)
        n.append(n[-1]+1)
        if np.sum(pn)<0.001 and np.sum(p_n)<0.001:
            #print(len(a))
            break 
    return n[-1]

def all_p(V,R1,R2,C1,C2,Q0,T,n):
    a=[]
    b=[]
    for i in range(n):
        an,bn=PN(V,R1,R2,C1,C2,Q0,T,i)
        a.append(an)
        b.append(bn)
    p0=1
    pn=[]
    p_n=[]
    for i in range(n):
        temp=np.zeros(n)
        temp[0:i+1]=1
        temp=temp.tolist()
        temp1=np.full(len(V),1.0)
        temp2=np.full(len(V),1.0)
        for j in range(n):
            if temp[j]<=0.1:
                pass
            else:
                temp1*=a[j]       
                temp2*=b[j]
        p0+=temp1+temp2
        pn.append(temp1)
        p_n.append(temp2)
    p0=1/p0
    pn=p0*np.array(pn)
    p_n=p0*np.array(p_n)
    return p0,pn,p_n

def G1n(V,R1,C1,C2,Q0,T,n):
    return Gamma1(V,R1,C1,C2,n,Q0,T)-Gamma1(-V,R1,C1,C2,-n,-Q0,T)

def G2n(V,R2,C1,C2,Q0,T,n):
    return Gamma2(V,R2,C1,C2,n,Q0,T)-Gamma2(-V,R2,C1,C2,-n,-Q0,T)

def current(V,R1,R2,C1,C2,Q0,T):
    n=check_p(V,R1,R2,C1,C2,Q0,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,T,n)
    I=p0*G1n(V,R1,C1,C2,Q0,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1n(V,R1,C1,C2,Q0,T,i)
        I+=p_n[i-1]*G1n(V,R1,C1,C2,Q0,T,-i)
    t=np.gradient(I)
    return t/np.sum(t)


def current2(V,R1,R2,C1,C2,Q0,T):
    n=check_p(V,R1,R2,C1,C2,Q0,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,T,n)
    I=p0*G2n(V,R2,C1,C2,Q0,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G2n(V,R2,C1,C2,Q0,T,i)
        I+=p_n[i-1]*G2n(V,R2,C1,C2,Q0,T,-i)
    t=np.gradient(I)
    return len(t)*t/np.sum(t)

## Faster proba

## High impedance

In [198]:
def fermi(T,x):
    return np.divide(1,1+np.exp(x/T))

def E1(C1,C2,V,n,Q0):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def E2(C1,C2,V,n,Q0):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def Gamma1(V,R1,C1,C2,n,Q0,T):
    k=C2/(C1+C2)
    Ec=(C1+C2)/(2*C1*C2)
    x=E1(C1,C2,V,n,Q0)-k**2*Ec
    return (x/(1-np.exp(-x/T)))/R1

def Gamma2(V,R2,C1,C2,n,Q0,T):
    k=C1/(C1+C2)
    Ec=(C1+C2)/(2*C1*C2)
    x=E2(C1,C2,V,-n,-Q0)-k**2*Ec
    return (x/(1-np.exp(-x/T)))/R2


def PN(V,R1,R2,C1,C2,Q0,T,n):
    mn=(Gamma1(V,R1,C1,C2,-n,Q0,T)+Gamma2(-V,R2,C1,C2,n,-Q0,T))/(Gamma1(-V,R1,C1,C2,n+1,-Q0,T)+Gamma2(V,R2,C1,C2,-n-1,Q0,T))
    en=(Gamma1(-V,R1,C1,C2,-n,-Q0,T)+Gamma2(V,R2,C1,C2,n,Q0,T))/(Gamma1(V,R1,C1,C2,n+1,Q0,T)+Gamma2(-V,R2,C1,C2,-n-1,-Q0,T))
    return en,mn

def check_p(V,R1,R2,C1,C2,Q0,T):
    Vmax=V
    n=[0]
    while True:
        a=[]
        b=[]
        for i in n:
            an,bn=PN(Vmax,R1,R2,C1,C2,Q0,T,i)
            a.append(an[0])
            b.append(bn[0])
        p0=1
        for i in range(len(a)):
            temp=np.zeros(len(a))
            temp[0:i+1]=1
            temp=temp.tolist()
            p0+=np.prod(a,where=temp)+np.prod(b,where=temp)
        p0=1/p0
        pn=p0*np.prod(a)
        p_n=p0*np.prod(b)
        #print(pn,p_n)
        n.append(n[-1]+1)
        if np.sum(pn)<0.001 and np.sum(p_n)<0.001:
            #print(len(a))
            break 
    return n[-1]

def all_p(V,R1,R2,C1,C2,Q0,T,n):
    a=[]
    b=[]
    for i in range(n):
        an,bn=PN(V,R1,R2,C1,C2,Q0,T,i)
        a.append(an)
        b.append(bn)
    p0=1
    pn=[]
    p_n=[]
    for i in range(n):
        temp=np.zeros(n)
        temp[0:i+1]=1
        temp=temp.tolist()
        temp1=np.full(len(V),1.0)
        temp2=np.full(len(V),1.0)
        for j in range(n):
            if temp[j]<=0.1:
                pass
            else:
                temp1*=a[j]       
                temp2*=b[j]
        p0+=temp1+temp2
        pn.append(temp1)
        p_n.append(temp2)
    p0=1/p0
    pn=p0*np.array(pn)
    p_n=p0*np.array(p_n)
    return p0,pn,p_n

def G1n(V,R1,C1,C2,Q0,T,n):
    return Gamma1(V,R1,C1,C2,n,Q0,T)-Gamma1(-V,R1,C1,C2,-n,-Q0,T)

def G2n(V,R2,C1,C2,Q0,T,n):
    return Gamma2(V,R2,C1,C2,n,Q0,T)-Gamma2(-V,R2,C1,C2,-n,-Q0,T)

def current(V,R1,R2,C1,C2,Q0,T):
    n=check_p(V,R1,R2,C1,C2,Q0,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,T,n)
    I=p0*G1n(V,R1,C1,C2,Q0,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1n(V,R1,C1,C2,Q0,T,i)
        I+=p_n[i-1]*G1n(V,R1,C1,C2,Q0,T,-i)
    t=np.gradient(I)
    return len(t)*t/np.sum(t)


def current2(V,R1,R2,C1,C2,Q0,T):
    n=check_p(V,R1,R2,C1,C2,Q0,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,T,n)
    I=p0*G2n(V,R2,C1,C2,Q0,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G2n(V,R2,C1,C2,Q0,T,i)
        I+=p_n[i-1]*G2n(V,R2,C1,C2,Q0,T,-i)
    t=np.gradient(I)
    return len(t)*t/np.sum(t)

## Experimental broadening

In [319]:
def conv_lockin(V,R1,R2,C1,C2,Q0,T,Vw):
    x=np.linspace(-Vw,Vw,len(V))
    a,b=np.meshgrid(V,x)
    conv=[]
    for i in V:
        f=current(i+x,R1,R2,C1,C2,Q0,T)
        g=(Vw**2-x**2)**(3/2)/(Vw**2)
        conv.append(np.sum(f*g))
    np.array(conv)
    out=np.gradient(conv)
    return out/np.sum(out)

In [956]:
x=np.linspace(-10.0,10.0,1000)
plt.plot(x,current(x,1,10000,0.3,0.01,0.0,0.1))
plt.plot(x,conv_lockin(x,1,10000,0.3,0.01,0.0,0.1,0.1))

## Low Impedance Superconductivity

In [981]:
def bcs(delta,x):
    eta=delta/10
    if delta<=0.01: 
        return 1
    else:
        return (np.sign(x))*np.imag(np.divide(np.abs(x+eta*1j),np.sqrt(delta-(x+eta*1j)**2)))

def fermi(T,x):
    return np.divide(1,1+np.exp(x/T))

def E1(C1,C2,V,n,Q0):
    k=C2/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def E2(C1,C2,V,n,Q0):
    k=C1/(C1+C2)
    return k*V+(n+Q0-1/2)/(C1+C2)

def Gamma1(V,R1,C1,C2,n,Q0,Delta,T):
    x=np.linspace(-4*np.max(V),4*np.max(V),7000)
    a,b=np.meshgrid(x,E1(C1,C2,V,n,Q0))
    t=a+b
    return np.dot( bcs(1.25,t)*(1-fermi(T,t)),bcs(Delta,x)*fermi(T,x) )/R1

def Gamma2(V,R2,C1,C2,n,Q0,Delta,T):
    x=np.linspace(-4*np.max(V),4*np.max(V),7000)
    a,b=np.meshgrid(x,E2(C1,C2,V,-n,-Q0))
    t=a+b
    return np.dot(1-fermi(T,t),bcs(Delta,x)*fermi(T,x))/R2


def PN(V,R1,R2,C1,C2,Q0,Delta,T,n):
    mn=(Gamma1(V,R1,C1,C2,-n,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,n,-Q0,Delta,T))/(Gamma1(-V,R1,C1,C2,n+1,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,-n-1,Q0,Delta,T))
    en=(Gamma1(-V,R1,C1,C2,-n,-Q0,Delta,T)+Gamma2(V,R2,C1,C2,n,Q0,Delta,T))/(Gamma1(V,R1,C1,C2,n+1,Q0,Delta,T)+Gamma2(-V,R2,C1,C2,-n-1,-Q0,Delta,T))
    return en,mn

def check_p(V,R1,R2,C1,C2,Q0,Delta,T):
    Vmax=V[-1]
    n=[0]
    while True:
        a=[]
        b=[]
        for i in n:
            an,bn=PN(Vmax,R1,R2,C1,C2,Q0,Delta,T,i)
            a.append(an[0])
            b.append(bn[0])
        p0=1
        for i in range(len(a)):
            temp=np.zeros(len(a))
            temp[0:i+1]=1
            temp=temp.tolist()
            p0+=np.prod(a,where=temp)+np.prod(b,where=temp)
        p0=1/p0
        pn=p0*np.prod(a)
        p_n=p0*np.prod(b)
        #print(pn,p_n)
        n.append(n[-1]+1)
        if pn<0.0001 and p_n<0.0001:
            #print(len(a))
            break 
    return n[-1]

def all_p(V,R1,R2,C1,C2,Q0,Delta,T,n):
    a=[]
    b=[]
    for i in range(n):
        an,bn=PN(V,R1,R2,C1,C2,Q0,Delta,T,i)
        a.append(an)
        b.append(bn)
    p0=1
    pn=[]
    p_n=[]
    for i in range(n):
        temp=np.zeros(n)
        temp[0:i+1]=1
        temp=temp.tolist()
        temp1=np.full(len(V),1.0)
        temp2=np.full(len(V),1.0)
        for j in range(n):
            if temp[j]<=0.1:
                pass
            else:
                temp1*=a[j]       
                temp2*=b[j]
        p0+=temp1+temp2
        pn.append(temp1)
        p_n.append(temp2)
    p0=1/p0
    pn=p0*np.array(pn)
    p_n=p0*np.array(p_n)
    return p0,pn,p_n

def G1n(V,R1,R2,C1,C2,Q0,Delta,T,n):
    return Gamma1(V,R1,C1,C2,n,Q0,Delta,T)-Gamma1(-V,R1,C1,C2,-n,-Q0,Delta,T)

def G2n(V,R1,R2,C1,C2,Q0,Delta,T,n):
    return Gamma2(V,R2,C1,C2,n,Q0,Delta,T)-Gamma2(-V,R2,C1,C2,-n,-Q0,Delta,T)

def currentD(V,R1,R2,C1,C2,Q0,Delta,T):
    n=check_p(V,R1,R2,C1,C2,Q0,Delta,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,Delta,T,n)
    I=p0*G1n(V,R1,R2,C1,C2,Q0,Delta,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G1n(V,R1,R2,C1,C2,Q0,Delta,T,i)
        I+=p_n[i-1]*G1n(V,R1,R2,C1,C2,Q0,Delta,T,-i)
    t=np.gradient(I)
    return t/np.sum(t)


def currentD2(V,R1,R2,C1,C2,Q0,Delta,T):
    n=check_p(V,R1,R2,C1,C2,Q0,Delta,T)
    p0,pn,p_n=all_p(V,R1,R2,C1,C2,Q0,Delta,T,n)
    I=p0*G2n(V,R1,R2,C1,C2,Q0,Delta,T,0)
    for i in range(1,n+1):
        I+=pn[i-1]*G2n(V,R1,R2,C1,C2,Q0,Delta,T,i)
        I+=p_n[i-1]*G2n(V,R1,R2,C1,C2,Q0,Delta,T,-i)
    t=np.gradient(I)
    return t/np.sum(t)


In [991]:

for i in range(9,len(dat)):
    print(i)
    spectra.load(Tfiles[i][T[i].index(1.3)])
    x=np.array(spectra.bias)
    y=np.array(spectra.conductance)
    y=y/np.sum(y)
    plt.plot(1000*x,y)
    par=sc.optimize.curve_fit(lambda xx, Q0,delta: currentD(xx,1000,1,C2[i],C1[i],Q0,delta,0.1), 1000*x, y,p0=[0.0,0.3],bounds=([-0.50,0.1],[0.5,1.3]))[0]
    a=currentD(1000*x,1000,1,C2[i],C1[i],par[0],par[1],0.1)
    plt.plot(1000*x,a)
    delta.append(par[1])
    Q0.append(par[0])

9
10
11
12


In [1003]:
i=7
spectra.load(Tfiles[i][T[i].index(1.3)])
x=np.array(spectra.bias)
y=np.array(spectra.conductance)
y=y/np.sum(y)
plt.plot(1000*x,y)
#a=currentD(1000*x,1000,1,C2[i],C1[i],Q0[i],delta[i],0.1)
#plt.plot(1000*x,a)

In [1094]:
temp1=[0.20538628915861032,0.3139708157071217,0.219155437068271,0.10000000000000002,0.9584898529396583,0.16640228862074904,0.3681922831136722,0.0,0.2097395898675521,0.458292984581787,0.10000000000000002,0.4714580736956538]
temp2=[373.0,578.0,419.0,363.0,865.0,846.0,873.0,305.0,613.0,1368.0,573.0,806.0]

plt.scatter(temp2,temp1)

In [1093]:
plt.scatter(size,C1)

# Adventure

# Josephson Pulse

In [1634]:
files=['S230803_738.dat','S230803_747.dat','S230803_753.dat','S230803_760.dat']
Pfiles=['C:/Users/jonor/Desktop/PhD/Lanak/Pb on Gr (analysis)/2023.07.26 - 1st evap/'+i for i in files]
Jfiles=['S230803_744.dat','S230803_752.dat','S230803_758.dat','S230803_764.dat']
PJfiles=['C:/Users/jonor/Desktop/PhD/Lanak/Pb on Gr (analysis)/2023.07.26 - 1st evap/'+i for i in Jfiles]


In [1693]:
ax1 = plt.subplot2grid((2,5), (0,0), colspan=2)
ax2 = plt.subplot2grid((2,5), (0,3), colspan=2)
ax3 = plt.subplot2grid((2,5), (1,0), colspan=2)
ax4 = plt.subplot2grid((2,5), (1,3), colspan=2)
ax=[ax1,ax2,ax3,ax4]
for i in range(len(PJfiles)):
    spectra.load(PJfiles[i])
    print(spectra.header.get('Lock-in>Amplitude'))
    x=np.array(spectra.bias)
    y=np.array(spectra.conductance)
    #a1=min(find_nearest(x,-0.0015),find_nearest(x,0.0015))
    #a2=max(find_nearest(x,-0.0015),find_nearest(x,0.0015))
    #ax[i].plot(x[a1:a2]/x[a1],y[a1:a2]/y[a1])
    a1=min(find_nearest(x,-0.0005),find_nearest(x,0.0005))
    a2=max(find_nearest(x,-0.0005),find_nearest(x,0.0005))
    y=np.array(spectra.current)
    ax[i].plot(x[a1:a2],y[a1:a2])
    ax[i].axhline(0.0)
    ax[i].vlines(0.8*150e-6,np.min(y[a1:a2]),np.max(y[a1:a2]))
    ax[i].vlines(-0.8*150e-6,np.min(y[a1:a2]),np.max(y[a1:a2]))




150E-6
150E-6
150E-6
150E-6


In [1691]:
spectra.load(PJfiles[0])


'150E-6'

In [1795]:
f= lambda x, a: 1/(x**2+a**2)
g= lambda x, Vw: np.sqrt(Vw**2-x**2+0.00001*1j)*np.heaviside(Vw-np.abs(x),0)
#h= lambda x, a: (-fermi(1,x)+fermi(1,-x))*(a*np.heaviside(x,0)+np.heaviside(-x,0))
h= lambda x, a: (a*np.heaviside(x,0))-1

Vw=0.1
t=0.1
x=np.linspace(-10,10,5000)
V=np.linspace(-10,10,5000)
y=f(V,t)
a,b=np.meshgrid(x,V)
conv=np.dot(g(a-b,Vw),f(x,0.1))
deconv=np.dot(np.linalg.pinv(g(a-b,Vw)),f(V,0.1))
plt.plot(V,y/np.sum(np.abs(y)))
plt.plot(V,conv/np.sum(np.abs(conv)))
plt.plot(x,deconv/np.sum(np.abs(deconv)))
#plt.axhline(0.0)
#plt.vlines(0.0,-0.001,0.001)


# Deconvoluting Oscillation

In [ ]:

conductance_new=savgol_filter(conductance_n,window,order)
x=np.linspace(min(bias_new),max(bias_new),n)
f2 = interp1d(bias_new, conductance_new, kind='cubic')
deconvolutionBias = np.arange(-energyR,energyR, spacing)

M_E, M_eV = np.meshgrid(deconvolutionBias,x)
M_N = dynes_curve(M_E,gap,dynesParameter)
M_NV = dynes_curve(M_E+M_eV,gap,dynesParameter)
M_dNV = dynes_curve_diff(M_E+M_eV,gap,dynesParameter)
M_f = fermiDirac(M_E, temperature)
M_fV = fermiDirac(M_E+M_eV, temperature)
M_dfV = fermiDirac_diff(M_E+M_eV, temperature)
M_g = M_dNV*(M_f-M_fV) - M_NV*M_dfV
M_g_pinv = np.linalg.pinv(M_g)
deconvolutionConductance = np.dot(M_g_pinv,f2(x))
return deconvolutionBias,deconvolutionConductance